In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"
import keras
from keras.models import load_model

# definte relu6
from tensorflow.python.keras import backend as K
def relu6(x):
    return K.relu(x, max_value=6)

# model = load_model('models/mobilenet_1.0_224_2018_06_18_16_08_08/face_attrib_mobilenet_1.0_224.10-0.18-0.17.hdf5', 
#                    custom_objects={'relu6': relu6})
# input_shape = (224, 224)
# model = load_model('models/mobilenet_1.0_192_2018_06_19_14_57_50/face_attrib_mobilenet_1.0_192.09-0.19-0.18.hdf5', 
#                    custom_objects={'relu6': relu6})
# input_shape = (192, 192)
# model = load_model('models/mobilenet_0.75_224_2018_06_20_11_40_04/face_attrib_mobilenet_0.75_224.11-0.19-0.17.hdf5', 
#                    custom_objects={'relu6': relu6})
# input_shape = (224, 224)
model = load_model('models/mobilenet_0.5_224_2018_06_20_17_04_49/face_attrib_mobilenet_0.5_224.14-0.19-0.18.hdf5', 
                   custom_objects={'relu6': relu6})
input_shape = (224, 224)

Using TensorFlow backend.


In [2]:
import os
import numpy as np
from data import load_attributes
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
data_dir = '/data/celeba'
val_images_path = os.path.join(data_dir, 'Img/img_align_celeba_crop_middle_val/')
test_images_path = os.path.join(data_dir, 'Img/img_align_celeba_crop_middle_test/')
attributes_path = os.path.join('.', 'celeba_header_lines_40.p')
multilabel_dict, labels = load_attributes(attributes_path)

def preprocess_image(image):
    image = image / 255.0
    image = image - 0.5
    image = image * 2.0
    return image

def get_data_in_X_Y(images_path, multilabel_dict):
    X = []
    Y = []
    for img_path in os.listdir(images_path):
        real_img_path = os.path.join(images_path, img_path)
        img = image.load_img(real_img_path, target_size=input_shape)
        x = image.img_to_array(img)
        x = preprocess_image(x)
        if img_path in multilabel_dict:
            X.append(x)
            Y.append(multilabel_dict[img_path])
    X=np.array(X)
    Y=np.array(Y)
    return X, Y
val_X, val_Y_gt = get_data_in_X_Y(val_images_path + 'all', multilabel_dict)
test_X, test_Y_gt = get_data_in_X_Y(test_images_path + 'all', multilabel_dict)

In [3]:
val_Y_predicted = model.predict(val_X, batch_size=64, verbose=1)
test_Y_predicted = model.predict(test_X, batch_size=64, verbose=1)
val_Y_predicted_rounded = np.round(val_Y_predicted)
test_Y_predicted_rounded = np.round(test_Y_predicted)

19962/19962 [==============================] - 22s 1ms/step


In [4]:
def get_accuracies(predicted, gt):
    assert predicted.shape == gt.shape 
    num_cols = len(predicted[0])
    num_rows = len(predicted)
    accuracies = []
    for i in range(num_cols):
        accuracies.append(0)
    for j in range(num_rows):
        for i in range(num_cols):
            if predicted[j][i] == gt[j][i]:
                accuracies[i] += 1
    for i in range(num_cols):
        accuracies[i] = accuracies[i] / 1.0 / num_rows
    return accuracies

val_accuracies = get_accuracies(val_Y_predicted_rounded, val_Y_gt)
print('-------------------- validation accuracies -------------------------')
print('mean: {:.04f}'.format(np.mean(val_accuracies)))
for i in range(len(labels)):
    print('{}, {:.04f}'.format(labels[i], val_accuracies[i]))
test_accuracies = get_accuracies(test_Y_predicted_rounded, test_Y_gt)
print('-------------------- test accuracies -------------------------')
print('mean: {:.04f}'.format(np.mean(test_accuracies)))
for i in range(len(labels)):
    print('{}, {:.04f}'.format(labels[i], test_accuracies[i]))
    


-------------------- validation accuracies -------------------------
mean: 0.9183
5_o_Clock_Shadow, 0.9421
Arched_Eyebrows, 0.8617
Attractive, 0.8166
Bags_Under_Eyes, 0.8418
Bald, 0.9897
Bangs, 0.9583
Big_Lips, 0.8460
Big_Nose, 0.8376
Black_Hair, 0.9117
Blond_Hair, 0.9571
Blurry, 0.9666
Brown_Hair, 0.8572
Bushy_Eyebrows, 0.9297
Chubby, 0.9561
Double_Chin, 0.9657
Eyeglasses, 0.9960
Goatee, 0.9682
Gray_Hair, 0.9795
Heavy_Makeup, 0.9273
High_Cheekbones, 0.8905
Male, 0.9871
Mouth_Slightly_Open, 0.9411
Mustache, 0.9628
Narrow_Eyes, 0.9376
No_Beard, 0.9619
Oval_Face, 0.7520
Pale_Skin, 0.9667
Pointy_Nose, 0.7769
Receding_Hairline, 0.9441
Rosy_Cheeks, 0.9517
Sideburns, 0.9743
Smiling, 0.9336
Straight_Hair, 0.8493
Wavy_Hair, 0.8440
Wearing_Earrings, 0.9167
Wearing_Hat, 0.9906
Wearing_Lipstick, 0.9259
Wearing_Necklace, 0.8810
Wearing_Necktie, 0.9539
Young, 0.8825
-------------------- test accuracies -------------------------
mean: 0.9141
5_o_Clock_Shadow, 0.9461
Arched_Eyebrows, 0.8408
Attractiv

In [16]:
new_images_path = os.path.join('/data/lfw/lfw_all_funneled_face_crop_l_0.3_r_0.3_t_0.4_d_0.2/all/')
new_attributes_path = os.path.join('.', 'lfw_header_lines_40.p')
new_multilabel_dict, new_labels = load_attributes(new_attributes_path)
new_X, new_Y_gt = get_data_in_X_Y(new_images_path, new_multilabel_dict)


In [17]:
new_Y_predicted = model.predict(new_X, batch_size=64, verbose=1)
new_Y_predicted_rounded = np.round(new_Y_predicted)

13143/13143 [==============================] - 22s 2ms/step


In [18]:
new_accuracies = get_accuracies(new_Y_predicted_rounded, new_Y_gt)
print('-------------------- test accuracies -------------------------')
print('mean: {:.04f}'.format(np.mean(new_accuracies)))
for i in range(len(labels)):
    print('{}, {:.04f}'.format(labels[i], new_accuracies[i]))

-------------------- test accuracies -------------------------
mean: 0.7384
5_o_Clock_Shadow, 0.6203
Arched_Eyebrows, 0.7698
Attractive, 0.6747
Bags_Under_Eyes, 0.6659
Bald, 0.9085
Bangs, 0.8598
Big_Lips, 0.6674
Big_Nose, 0.5404
Black_Hair, 0.8745
Blond_Hair, 0.9610
Blurry, 0.8304
Brown_Hair, 0.6669
Bushy_Eyebrows, 0.5291
Chubby, 0.6743
Double_Chin, 0.6810
Eyeglasses, 0.9216
Goatee, 0.7778
Gray_Hair, 0.8535
Heavy_Makeup, 0.9340
High_Cheekbones, 0.8401
Male, 0.9781
Mouth_Slightly_Open, 0.7860
Mustache, 0.9101
Narrow_Eyes, 0.4376
No_Beard, 0.7962
Oval_Face, 0.4810
Pale_Skin, 0.5002
Pointy_Nose, 0.3336
Receding_Hairline, 0.5225
Rosy_Cheeks, 0.8064
Sideburns, 0.7043
Smiling, 0.8949
Straight_Hair, 0.5862
Wavy_Hair, 0.5825
Wearing_Earrings, 0.8934
Wearing_Hat, 0.8974
Wearing_Lipstick, 0.9301
Wearing_Necklace, 0.8057
Wearing_Necktie, 0.6875
Young, 0.7526
